In [ ]:
!pip install xgboost

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#from xgboost import XGBRegressor

import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Course_work/notebooks/data/df_mol2vec.csv')
df.drop(columns = ['Unnamed: 0'], inplace = True)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,292,293,294,295,296,297,298,299,pdbcode,affinity
0,-33.215520,-4.911912,-58.069412,-21.501130,86.036830,-48.711243,-163.09206,-56.392216,177.12903,-0.203314,...,118.41117,87.59636,-135.84192,-56.251205,12.598450,-148.720890,-178.94377,-31.278870,3zzf,0.40
1,-49.548927,-5.533656,-78.656740,11.228692,96.537285,-45.226390,-198.04187,-50.896503,194.37296,13.702780,...,160.35870,97.12160,-164.78786,-62.422604,18.535614,-147.918760,-208.95258,-46.084557,3gww,0.45
2,-23.400223,-7.932979,-35.621426,-2.120256,59.704235,-22.519524,-114.47008,-34.282997,111.38798,7.034303,...,94.77718,61.17864,-83.11074,-31.502813,7.987552,-95.268074,-120.75750,-28.159447,1w8l,0.49
3,-26.764908,-7.937400,-49.278860,-9.648706,69.367480,-31.425886,-146.40346,-37.342216,129.86770,14.527238,...,105.85985,69.64447,-126.76569,-49.138590,1.389015,-115.341490,-158.34018,-35.511227,3fqa,0.49
4,-33.097664,-24.753980,-71.645750,17.423645,76.952320,-45.259354,-184.55115,-47.502502,176.07903,4.592343,...,142.47672,75.98463,-151.25670,-62.206776,0.313482,-147.893430,-202.49968,-42.562164,1zsb,0.60


In [4]:
x = df.drop(columns = ['affinity', 'pdbcode']).copy()
y = df.affinity.copy()

## RFR

squared, None, 100: 1.1208997140876742 1.4205143749758993

squared, None, 200: 1.1144450156834937 1.4067854577737786

squared, 3, 100: 1.315556467918127 1.6494418463146427

squared, 3, 200: 1.2853657198683157 1.595820864908951

squared, 6, 100: 1.262208942814106 1.589164972060358

squared, 6, 200: 1.234568788129492 1.539395192420277

absolute_error, None, 200: 

absolute_error, 3, 100: 

absolute_error, 3, 200: 

absolute_error, 6, 100: 

absolute_error, 6, 200: 

In [5]:
params = {
    'criterion' : ['squared_error', 'absolute_error'],
    'max_depth' : [None, 3, 6],
    'n_estimators' : [100, 200]
}

In [6]:
x_train, x_val, y_train, y_val = train_test_split(x, y, train_size=0.8)

In [7]:
model = RandomForestRegressor(random_state=0, criterion =  'absolute_error', max_depth = 3, n_estimators = 100)

In [8]:
model.fit(x_train, y_train)

KeyboardInterrupt: ignored

In [ ]:
pred_val = model.predict(x_val)

print('mae, rmse:')
print(mean_absolute_error(y_val, pred_val), mean_squared_error(y_val, pred_val, squared = False))

## GS

In [ ]:
model = XGBRegressor()
params = {
    'learning_rate' : [0.1, 0.01, 0.001],
    'max_depth' : [3, 6],
    'n_estimators' : [100, 200],
    'n_jobs' : [0]
}

In [ ]:
gs = GridSearchCV(model, param_grid=params, scoring='neg_mean_absolute_error', cv=5)
gs.fit(x, y)
gs.best_params_S

[18:28:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:28:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:28:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:29:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:29:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:30:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:31:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:31:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 200, 'n_jobs': 0}

In [ ]:
sorted(metrics.SCORERS.keys())